In [4]:
import tensorflow as tf
import re
import numpy as np

### Get the corpus

In [5]:
corpus = [
    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',
    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

### Create vocabulary

In [6]:
vocab = set(re.sub(r' +', ' ', ' '.join(corpus)).strip().split(' '))
vocab

{'Exceptionally',
 'I',
 'It',
 'Its',
 'Never',
 'The',
 'This',
 'Wonderful',
 'Wonderfully',
 'You',
 'a',
 'acting',
 'an',
 'and',
 'boring',
 'brillent',
 'brilliant',
 'did',
 'directed',
 'excellent',
 'executed',
 'fantastic',
 'good',
 'horrible',
 'is',
 'it',
 'like',
 'money',
 'move',
 'movie',
 'not',
 'of',
 'pathetic',
 'picture',
 'recommend',
 'series',
 'should',
 'such',
 'the',
 'very',
 'was',
 'waste',
 'watch',
 'watched',
 'will'}

In [7]:
word2id = {word:i for i, word in enumerate(vocab)}
word2id

{'did': 0,
 'such': 1,
 'will': 2,
 'It': 3,
 'it': 4,
 'picture': 5,
 'Wonderful': 6,
 'watch': 7,
 'like': 8,
 'I': 9,
 'watched': 10,
 'brillent': 11,
 'money': 12,
 'very': 13,
 'recommend': 14,
 'good': 15,
 'Its': 16,
 'pathetic': 17,
 'You': 18,
 'should': 19,
 'move': 20,
 'Exceptionally': 21,
 'a': 22,
 'movie': 23,
 'not': 24,
 'executed': 25,
 'fantastic': 26,
 'is': 27,
 'boring': 28,
 'Wonderfully': 29,
 'and': 30,
 'directed': 31,
 'excellent': 32,
 'of': 33,
 'the': 34,
 'Never': 35,
 'This': 36,
 'was': 37,
 'an': 38,
 'acting': 39,
 'horrible': 40,
 'waste': 41,
 'brilliant': 42,
 'series': 43,
 'The': 44}

In [8]:
id2word = list(vocab)
id2word

['did',
 'such',
 'will',
 'It',
 'it',
 'picture',
 'Wonderful',
 'watch',
 'like',
 'I',
 'watched',
 'brillent',
 'money',
 'very',
 'recommend',
 'good',
 'Its',
 'pathetic',
 'You',
 'should',
 'move',
 'Exceptionally',
 'a',
 'movie',
 'not',
 'executed',
 'fantastic',
 'is',
 'boring',
 'Wonderfully',
 'and',
 'directed',
 'excellent',
 'of',
 'the',
 'Never',
 'This',
 'was',
 'an',
 'acting',
 'horrible',
 'waste',
 'brilliant',
 'series',
 'The']

### one hot encoding for word

In [9]:
def ohe(word, word2id, dim=45):
    vec = [0]*dim
    vec[word2id[word]] = 1
    return vec

### Create training data

In [10]:
def create_training_data_from_a_sentence(words, window_size):
    ans = []
    lookup = (window_size-1)//2
    for i in range(len(words)):
        if (i-lookup) >= 0 and (i+lookup) < len(words):
            x = []
            j = i-lookup
            while j <= (i+lookup):
                if j != i:
                    x.append(words[j])
                j += 1
            ans.append([x, words[i]])
    return ans

window_size = 3 # keep it odd
training_data = [] # [[x1,x2],y1]
for i, line in enumerate(corpus):
    words = line.split(' ')
    training_data.extend(create_training_data_from_a_sentence(words, window_size))

training_data

[[['This', 'an'], 'is'],
 [['is', 'excellent'], 'an'],
 [['an', 'movie'], 'excellent'],
 [['The', 'was'], 'move'],
 [['move', 'fantastic'], 'was'],
 [['was', 'I'], 'fantastic'],
 [['fantastic', 'like'], 'I'],
 [['I', 'it'], 'like'],
 [['You', 'watch'], 'should'],
 [['should', 'it'], 'watch'],
 [['watch', 'is'], 'it'],
 [['it', 'brilliant'], 'is'],
 [['Wonderfully', 'and'], 'directed'],
 [['directed', 'executed'], 'and'],
 [['and', 'I'], 'executed'],
 [['executed', 'like'], 'I'],
 [['I', 'it'], 'like'],
 [['Its', 'fantastic'], 'a'],
 [['a', 'series'], 'fantastic'],
 [['Never', 'such'], 'watched'],
 [['watched', 'a'], 'such'],
 [['such', 'brillent'], 'a'],
 [['a', 'movie'], 'brillent'],
 [['It', 'a'], 'is'],
 [['is', 'Wonderful'], 'a'],
 [['a', 'movie'], 'Wonderful'],
 [['waste', 'money'], 'of'],
 [['It', 'very'], 'was'],
 [['was', 'boring'], 'very'],
 [['I', 'not'], 'did'],
 [['did', 'like'], 'not'],
 [['not', 'the'], 'like'],
 [['like', 'movie'], 'the'],
 [['The', 'was'], 'movie'],
 [[

In [11]:
from functools import reduce

X  = []
Y = []

for x, y in training_data:
    Y.append(ohe(y, word2id, 45))
    X.append(list(reduce(lambda u, v: [i|j for i, j in zip(u, v)], [ohe(word, word2id, 45) for word in x])))

In [12]:
len(X), len(Y), len(X[0]), len(Y[0])

(39, 39, 45, 45)

In [13]:
# sanity check of ith sample
i = 13
[[id2word[j] for j in range(len(X[i])) if X[i][j] == 1], [id2word[j] for j in range(len(Y[i])) if Y[i][j] == 1]]


[['executed', 'directed'], ['and']]

In [14]:
X_train = np.array(X)
Y_train = np.array(Y)
X_train, Y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [15]:
X_train.shape, Y_train.shape

((39, 45), (39, 45))

### Neural network

In [16]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation='linear', input_shape=[45]),
  tf.keras.layers.Dense(45)
])

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=500)

Epoch 1/500
2/2 [==============================] - 0s 4ms/step - loss: 3.8038 - accuracy: 0.0513
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 3.7951 - accuracy: 0.0513
Epoch 3/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7877 - accuracy: 0.0513
Epoch 4/500
2/2 [==============================] - 0s 5ms/step - loss: 3.7808 - accuracy: 0.0513
Epoch 5/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7740 - accuracy: 0.0513
Epoch 6/500
2/2 [==============================] - 0s 5ms/step - loss: 3.7671 - accuracy: 0.0513
Epoch 7/500
2/2 [==============================] - 0s 5ms/step - loss: 3.7606 - accuracy: 0.0769
Epoch 8/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7541 - accuracy: 0.0769
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 3.7477 - accuracy: 0.0769
Epoch 10/500
2/2 [==============================] - 0s 2ms/step - loss: 3.7414 - accuracy: 0.0769
Epoch 11/500
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 3.1813 - accuracy: 0.4359
Epoch 85/500
2/2 [==============================] - 0s 998us/step - loss: 3.1723 - accuracy: 0.4359
Epoch 86/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1629 - accuracy: 0.4359
Epoch 87/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1534 - accuracy: 0.4359
Epoch 88/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1439 - accuracy: 0.4359
Epoch 89/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1342 - accuracy: 0.4359
Epoch 90/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1245 - accuracy: 0.4103
Epoch 91/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1148 - accuracy: 0.4103
Epoch 92/500
2/2 [==============================] - 0s 2ms/step - loss: 3.1050 - accuracy: 0.4103
Epoch 93/500
2/2 [==============================] - 0s 2ms/step - loss: 3.0947 - accuracy: 0.4103
Epoch 94/500
2/2 [=============

2/2 [==============================] - 0s 1ms/step - loss: 2.3588 - accuracy: 0.5385
Epoch 167/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3492 - accuracy: 0.5385
Epoch 168/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3395 - accuracy: 0.5385
Epoch 169/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3300 - accuracy: 0.5385
Epoch 170/500
2/2 [==============================] - 0s 922us/step - loss: 2.3198 - accuracy: 0.5641
Epoch 171/500
2/2 [==============================] - 0s 2ms/step - loss: 2.3102 - accuracy: 0.5641
Epoch 172/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3005 - accuracy: 0.5641
Epoch 173/500
2/2 [==============================] - 0s 1ms/step - loss: 2.2908 - accuracy: 0.5641
Epoch 174/500
2/2 [==============================] - 0s 926us/step - loss: 2.2810 - accuracy: 0.5641
Epoch 175/500
2/2 [==============================] - 0s 2ms/step - loss: 2.2714 - accuracy: 0.5641
Epoch 176/500
2/2 [=

2/2 [==============================] - 0s 2ms/step - loss: 1.6099 - accuracy: 0.7436
Epoch 249/500
2/2 [==============================] - 0s 2ms/step - loss: 1.6017 - accuracy: 0.7436
Epoch 250/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5934 - accuracy: 0.7692
Epoch 251/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5855 - accuracy: 0.7692
Epoch 252/500
2/2 [==============================] - 0s 1ms/step - loss: 1.5770 - accuracy: 0.7692
Epoch 253/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5692 - accuracy: 0.7692
Epoch 254/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5612 - accuracy: 0.7949
Epoch 255/500
2/2 [==============================] - 0s 1ms/step - loss: 1.5532 - accuracy: 0.7949
Epoch 256/500
2/2 [==============================] - 0s 2ms/step - loss: 1.5455 - accuracy: 0.7949
Epoch 257/500
2/2 [==============================] - 0s 1ms/step - loss: 1.5376 - accuracy: 0.7949
Epoch 258/500
2/2 [=====

2/2 [==============================] - 0s 1ms/step - loss: 1.0525 - accuracy: 0.8205
Epoch 331/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0472 - accuracy: 0.8205
Epoch 332/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0421 - accuracy: 0.8205
Epoch 333/500
2/2 [==============================] - 0s 986us/step - loss: 1.0369 - accuracy: 0.8205
Epoch 334/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0317 - accuracy: 0.8205
Epoch 335/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0268 - accuracy: 0.8205
Epoch 336/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0218 - accuracy: 0.8205
Epoch 337/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0168 - accuracy: 0.8205
Epoch 338/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0117 - accuracy: 0.8205
Epoch 339/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0070 - accuracy: 0.8462
Epoch 340/500
2/2 [===

2/2 [==============================] - 0s 1ms/step - loss: 0.7068 - accuracy: 0.8718
Epoch 413/500
2/2 [==============================] - 0s 1ms/step - loss: 0.7033 - accuracy: 0.8718
Epoch 414/500
2/2 [==============================] - ETA: 0s - loss: 0.7231 - accuracy: 0.84 - 0s 997us/step - loss: 0.6999 - accuracy: 0.8718
Epoch 415/500
2/2 [==============================] - 0s 499us/step - loss: 0.6967 - accuracy: 0.8718
Epoch 416/500
2/2 [==============================] - 0s 1ms/step - loss: 0.6934 - accuracy: 0.8718
Epoch 417/500
2/2 [==============================] - 0s 1ms/step - loss: 0.6901 - accuracy: 0.8718
Epoch 418/500
2/2 [==============================] - 0s 1ms/step - loss: 0.6867 - accuracy: 0.8718
Epoch 419/500
2/2 [==============================] - 0s 1ms/step - loss: 0.6838 - accuracy: 0.8718
Epoch 420/500
2/2 [==============================] - 0s 2ms/step - loss: 0.6806 - accuracy: 0.8718
Epoch 421/500
2/2 [==============================] - 0s 902us/step - loss: 0.

2/2 [==============================] - 0s 2ms/step - loss: 0.4970 - accuracy: 0.8974
Epoch 495/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4952 - accuracy: 0.8974
Epoch 496/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4932 - accuracy: 0.8974
Epoch 497/500
2/2 [==============================] - 0s 2ms/step - loss: 0.4914 - accuracy: 0.8974
Epoch 498/500
2/2 [==============================] - 0s 998us/step - loss: 0.4894 - accuracy: 0.8974
Epoch 499/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4876 - accuracy: 0.8974
Epoch 500/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4857 - accuracy: 0.8974


In [17]:
w, b = model.layers[0].get_weights()
w.shape, b.shape

((45, 10), (10,))

### word embedding of ith training target

In [18]:
i = 25
print(f'word embedding of "{training_data[i][1]}" = {np.dot(X_train[i:(i+1)], w) + b}')

word embedding of "Wonderful" = [[ 1.24432588 -1.99069145 -1.0718352   0.09638837  0.70506335 -0.69508408
  -1.16809948  1.61583742  0.05783282 -0.37892926]]


### get word to embedding mapping

In [36]:
words = list(map(lambda x : x[1], training_data))
words[:5], len(words)

(['is', 'an', 'excellent', 'move', 'was'], 39)

In [39]:
import pandas as pd
df = pd.DataFrame(np.dot(X_train, w) + b)
df.index = words
df

,0,1,2,3,4,5,6,7,8,9
is,1.279979,-0.168190,0.830259,-1.351963,1.281968,-0.693271,-1.392339,0.606229,1.036305,0.439766
an,0.486860,-1.254338,-1.101827,0.904674,-1.031337,-0.531188,-1.630290,-0.570660,1.106059,0.412546
excellent,0.943121,-1.230062,-1.163954,-0.355245,1.236271,-0.037581,-1.106240,0.614156,1.299751,1.332984
move,1.262650,-1.626576,-1.098237,-1.685319,1.158950,1.234595,0.690524,-0.635415,-0.811159,-1.013804
was,2.073212,-0.482738,0.414404,-0.393205,-1.195484,1.057598,-0.312190,1.046732,-0.351269,1.011717
fantastic,-0.125868,-0.987034,-0.892619,-1.352716,0.724098,-0.408413,-0.561697,0.590947,-1.754664,-0.287357
I,1.495555,0.459882,-0.693218,0.799863,-0.342810,1.351781,0.193604,1.593190,-0.082925,1.054695
like,-0.479119,-0.145885,0.817430,-1.168349,0.993559,-1.390241,-1.301019,1.848546,-1.644978,1.061836
should,0.467065,0.853717,-1.323212,-0.346991,0.601265,1.220424,-0.572727,1.608496,1.524196,-0.734867
watch,0.154833,0.239015,1.757576,-0.263729,1.068222,0.016080,-0.157643,1.234838,-1.576562,1.459731


In [74]:
# average out representation of each word
df = df.groupby(df.index).mean()
df.shape

(27, 10)

### visualize embeddings in tensorflow projector
http://projector.tensorflow.org/

upload metatdata.tsv and vectors.tsv to visualize

In [75]:
%reload_ext tensorboard
import os
logdir = "logs/visualize_embedding"

In [76]:
with open(f'{logdir}/metadata.tsv', 'w') as f:
    f.write('\n'.join(df.index))

In [77]:
df.to_csv(f'{logdir}/vectors.tsv', index=False, sep='\t', header=False)

### the output was boring :(
### Let's try with more data

In [172]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [263]:
data = tfds.load('imdb_reviews/plain_text', split='unsupervised')
data = data.shuffle(1024)
it = iter(data)

reviews = []
examples = 100
for i in range(examples):
    review = it.next()
    reviews.append(review['text'].numpy().decode('utf-8'))

In [294]:
def clean_a_sentence(sentence):
    import re
    sentence = re.sub(r'<br />', ' ', sentence)
    sentence = re.sub(r'[, ]+', ' ', sentence).strip()
    return sentence

In [297]:
sentences = []
for review in reviews:
    sentences.extend(list(filter(lambda line: line != '', map(lambda line: clean_a_sentence(line), review.split('.')))))
sentences = [sentence.split(' ') for sentence in sentences]

In [298]:
sentences[0]

['You',
 'know',
 "it's",
 'like',
 'a',
 'Spaghetti',
 'Western',
 'but',
 'with',
 'ray-guns']

In [301]:
vocab = set([word for sen in sentences for word in sen])

In [303]:
len(vocab)

5744

In [304]:
word2id = {word:i for i, word in enumerate(vocab)}
id2word = list(vocab)

In [305]:
window_size = 3 # keep it odd
training_data = [] # [[x1,x2],y1]
for i, sentence in enumerate(sentences):
    training_data.extend(create_training_data_from_a_sentence(sentence, window_size))

In [308]:
training_data[:4], len(training_data)

([[['You', "it's"], 'know'],
  [['know', 'like'], "it's"],
  [["it's", 'a'], 'like'],
  [['like', 'Spaghetti'], 'a']],
 19993)

In [312]:
train_X = [[word2id[j] for j in i[0]] for i in training_data]
train_X = np.array(train_X)
train_X

array([[  24, 4323],
       [2765,  958],
       [4323, 4316],
       ...,
       [4323, 4316],
       [4610, 3148],
       [4316, 3762]])

In [315]:
train_Y = [[word2id[i[1]]] for i in training_data]
train_Y = np.array(train_Y)
train_Y

array([[2765],
       [4323],
       [ 958],
       ...,
       [4610],
       [4316],
       [3148]])

In [316]:
train_X.shape, train_Y.shape

((19993, 2), (19993, 1))

## this time using built in embedding layer

In [319]:
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [334]:
embedding_dim = 16
model = Sequential([Embedding(input_dim=len(vocab), output_dim=embedding_dim),
                   GlobalAveragePooling1D(),
                   Dense(len(vocab))])

In [342]:
model.compile(Adam(0.0005), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [336]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          91904     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5744)              97648     
Total params: 189,552
Trainable params: 189,552
Non-trainable params: 0
_________________________________________________________________


In [344]:
# I ran model fit multiple times with different epochs and learning rate
# so don't think it started with 71% accuracy (instead it started with 5%)

In [343]:
history = model.fit(train_X, train_Y, epochs=10)

Epoch 1/10
625/625 [==============================] - 2s 4ms/step - loss: 1.1532 - accuracy: 0.7141
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 1.1483 - accuracy: 0.7151
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 1.1475 - accuracy: 0.7145
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 1.1468 - accuracy: 0.7150
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 1.1461 - accuracy: 0.7154
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 1.1456 - accuracy: 0.7148
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 1.1450 - accuracy: 0.7149
Epoch 8/10
625/625 [==============================] - 2s 4ms/step - loss: 1.1445 - accuracy: 0.7153
Epoch 9/10
625/625 [==============================] - 2s 4ms/step - loss: 1.1440 - accuracy: 0.7156
Epoch 10/10
625/625 [==============================] - 3s 4ms/step - loss: 1.1435 - accuracy: 0.7170

In [401]:
df = pd.DataFrame(model.layers[1](model.layers[0](train_X)).numpy())

In [402]:
df.index = list(map(lambda x : id2word[x], train_Y.squeeze()))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
know,-1.568521,2.159748,0.073384,0.697752,-2.365087,1.776173,-2.214857,-1.254740,-0.276653,0.482874,1.926827,-0.159665,0.721765,-1.239955,0.941057,-1.453552
it's,1.219179,-1.624454,-0.584512,-0.441756,-1.670563,-0.200357,-1.627250,-0.691518,-0.184850,0.100584,0.852530,-0.432806,-1.578029,-0.864022,0.102814,0.621504
like,-0.771871,0.139178,1.307506,-0.241945,0.452467,-0.584545,0.383954,-0.604088,-0.184459,-1.163455,0.934987,2.023859,0.721423,-0.588926,-0.190456,-1.564948
a,-1.632095,-2.473989,-2.767762,0.265900,-2.337109,-0.610698,-0.703219,1.637693,-1.570144,-0.175825,-4.160265,2.142572,-3.034826,-0.165649,2.854442,0.478558
Spaghetti,-3.633009,2.471024,-1.740682,-0.981225,0.150124,1.703184,1.680602,-2.388871,1.063939,1.032531,0.074336,-0.468216,1.887318,-2.759635,0.090123,-3.745395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
current,-0.176891,1.042812,1.305274,-0.822389,-1.419245,0.360981,1.518140,-0.964185,-0.819043,0.443630,0.728262,0.831522,0.604441,0.418223,-0.888275,-0.229456
it's,1.698527,-3.353550,-2.800743,-0.441903,-3.601066,1.238906,-2.870667,-2.572163,-1.608111,1.117736,-1.851267,0.050666,-0.454642,0.484064,-0.713263,0.077774
not,-0.771871,0.139178,1.307506,-0.241945,0.452467,-0.584545,0.383954,-0.604088,-0.184459,-1.163455,0.934987,2.023859,0.721423,-0.588926,-0.190456,-1.564948
a,-1.400604,-0.996924,-0.616170,-1.156898,-2.048256,1.007186,-0.675739,0.967947,-0.284731,1.763657,-1.291728,0.588641,-0.448475,-1.043121,1.432992,1.248252


In [403]:
# average out representation of each word
df = df.groupby(df.index).mean()
df.shape # < vocab size which was expected - why < - first and last words of each sentence are not considered for Y var

(5126, 16)

In [411]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
!,0.614773,2.184768,-1.875291,-2.392619,-1.983358,-0.466332,-1.477482,0.724128,1.692778,1.414513,1.626997,-0.187993,1.396734,0.893966,0.031648,0.464041
!),-1.242826,0.632045,-0.905323,-1.453357,-2.560370,-2.671518,-0.353624,1.379483,-0.828765,0.013821,1.080674,0.351833,0.339504,0.983073,0.977823,-0.570352
"""",0.348271,1.808484,-2.669932,-0.228845,1.090854,-1.378595,-1.647741,-1.753770,-0.303282,0.836549,2.544893,-2.430078,1.735083,-0.380034,-1.728032,-1.318443
"""Ai!",2.715514,-2.795877,-2.191293,0.957296,2.489571,4.871963,0.480067,4.701493,-0.834875,2.381193,-1.849827,1.158584,4.815081,0.151489,0.532469,4.825199
"""American",1.888345,2.215180,0.214737,2.237525,3.116467,0.615493,2.369236,-1.580535,-1.097147,1.000498,2.292800,3.216928,0.400558,1.796580,-5.076876,2.271558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zesty,-1.665819,-0.860462,-1.023836,-1.570753,0.759116,-0.494996,1.400812,0.725497,0.829708,-1.468480,0.229226,1.561531,-0.804258,-0.340858,1.710068,0.819719
zombiefied,-2.177390,-1.137583,-3.040695,-0.101057,1.404633,3.883577,-1.636874,0.432281,-0.799712,1.516297,1.715312,2.050118,3.725192,-3.073760,2.624060,3.469141
zombies,3.182648,-2.354931,3.487257,-0.163048,2.256392,2.726834,2.055700,2.510818,-0.161862,2.594480,0.615106,-1.619133,-2.938797,0.928330,-4.164789,-1.487806
~,-1.982251,0.917610,0.091327,-0.909476,2.679627,2.147624,3.226024,-0.264156,-1.890371,-2.483286,0.406137,-4.496862,2.493783,2.942935,1.256528,-2.970675


In [410]:
pd.DataFrame({'index':df.index.values}).to_csv(f'{logdir}/metadata.tsv', index=False, sep='\t', header=False)
df.to_csv(f'{logdir}/vectors.tsv', index=False, sep='\t', header=False)